In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import train_test_split

In [2]:
torch.manual_seed(42)

In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
data=pd.read_csv('/content/online_shoppers_intention.csv')
data.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [5]:
data.shape

(12330, 18)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

In [7]:
data.isnull().sum()

,0
Administrative,0
Administrative_Duration,0
Informational,0
Informational_Duration,0
ProductRelated,0
ProductRelated_Duration,0
BounceRates,0
ExitRates,0
PageValues,0
SpecialDay,0


In [8]:
X=data.drop('Revenue',axis=1)
y=data['Revenue']

In [9]:
y.replace({'False.':0,'True.':1},inplace=True)
X['VisitorType'].replace({'Returning_Visitor':0,'New_Visitor':1,'Other':2},inplace=True)

<ipython-input-9-d6445f0a482e>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['VisitorType'].replace({'Returning_Visitor':0,'New_Visitor':1,'Other':2},inplace=True)
<ipython-input-9-d6445f0a482e>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X['VisitorType'].replace({'Returning_Visitor':0,'Ne

In [10]:
X['Month'].replace({'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'June':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12},inplace=True)

<ipython-input-10-4c001699778e>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['Month'].replace({'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'June':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12},inplace=True)
<ipython-input-10-4c001699778e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X['

In [11]:
X

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend
0,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.000000,0.0,2,1,1,1,1,0,False
1,0,0.0,0,0.0,2,64.000000,0.000000,0.100000,0.000000,0.0,2,2,2,1,2,0,False
2,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.000000,0.0,2,4,1,9,3,0,False
3,0,0.0,0,0.0,2,2.666667,0.050000,0.140000,0.000000,0.0,2,3,2,2,4,0,False
4,0,0.0,0,0.0,10,627.500000,0.020000,0.050000,0.000000,0.0,2,3,3,1,4,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,3,145.0,0,0.0,53,1783.791667,0.007143,0.029031,12.241717,0.0,12,4,6,1,1,0,True
12326,0,0.0,0,0.0,5,465.750000,0.000000,0.021333,0.000000,0.0,11,3,2,1,8,0,True
12327,0,0.0,0,0.0,6,184.250000,0.083333,0.086667,0.000000,0.0,11,3,2,1,13,0,True
12328,4,75.0,0,0.0,15,346.000000,0.000000,0.021053,0.000000,0.0,11,2,2,3,11,0,False


In [12]:
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [13]:
#creating a custom class
import numpy as np
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    # Convert features to NumPy array with the correct data type before creating a tensor
    self.features=torch.tensor(features.values.astype(np.float32),dtype=torch.float32) # CHANGED: This line casts the features to float32
    self.labels=torch.tensor(labels.values,dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self,idx):
    return self.features[idx],self.labels[idx]

In [14]:
#creating object for training and testing
train_dataset=CustomDataset(X_train,Y_train)
test_dataset=CustomDataset(X_test,Y_test)

In [15]:
#creating train and test loader
trainloader=DataLoader(train_dataset,batch_size=64,shuffle=True,pin_memory=True)
testloader=DataLoader(test_dataset,batch_size=64,shuffle=False,pin_memory=True)

In [16]:
len(trainloader)

155

In [17]:
len(testloader)

39

In [18]:
#custom nn module
class myNN(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.model=nn.Sequential(
      nn.Linear(num_features,128),
      nn.ReLU(),
      nn.Linear(128,64),
      nn.ReLU(),
      nn.Linear(64,2)
    )
  def forward(self,x):
    return self.model(x)



In [19]:
learning_rate=0.01
epochs=25

In [20]:
#instatiate the model
model=myNN(X_train.shape[1])
model=model.to(device)
#loss function
criterion=nn.CrossEntropyLoss()
#optimizer
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [21]:
#training loop
for epoch in range(epochs):
  total_epoch_loss=0
  for batch_features,batch_labels in trainloader:
    #move data to gpu
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)
    #forward pass
    output=model(batch_features)
    #calculate loss
    loss=criterion(output,batch_labels)
    #back pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_epoch_loss+=loss.item()
  print(f"Epoch {epoch+1}/{epochs}, Loss: {total_epoch_loss/len(trainloader)}")


Epoch 1/25, Loss: 2.114789637442558
Epoch 2/25, Loss: 0.3361316764546979
Epoch 3/25, Loss: 0.32906334784723096
Epoch 4/25, Loss: 0.3209966222124715
Epoch 5/25, Loss: 0.30503707643478145
Epoch 6/25, Loss: 0.29791420304006144
Epoch 7/25, Loss: 0.30373653764686276
Epoch 8/25, Loss: 0.2997825334629705
Epoch 9/25, Loss: 0.30741076195432293
Epoch 10/25, Loss: 0.2896590967332163
Epoch 11/25, Loss: 0.3118458938694769
Epoch 12/25, Loss: 0.3184171098855234
Epoch 13/25, Loss: 0.2882252960916488
Epoch 14/25, Loss: 0.28503127525891025
Epoch 15/25, Loss: 0.28089354038238523
Epoch 16/25, Loss: 0.2817846720257113
Epoch 17/25, Loss: 0.27620720546091754
Epoch 18/25, Loss: 0.27946840284332153
Epoch 19/25, Loss: 0.2816275819174705
Epoch 20/25, Loss: 0.2709938589363329
Epoch 21/25, Loss: 0.28245467870466173
Epoch 22/25, Loss: 0.27703447654362645
Epoch 23/25, Loss: 0.2738705216876922
Epoch 24/25, Loss: 0.2792235472509938
Epoch 25/25, Loss: 0.27523245417302655


In [22]:
#setting the model to evaluate mode
model.eval()

myNN(
  (model): Sequential(
    (0): Linear(in_features=17, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [23]:
#setting the model to evaluate mode
model.eval()

myNN(
  (model): Sequential(
    (0): Linear(in_features=17, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [24]:
#evaluation
total=0
correct=0
with torch.no_grad():
  for batch_features,batch_labels in testloader:
    #move data to gpu
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    outputs=model(batch_features)
    _,predicted=torch.max(outputs.data,1)
    total+=batch_labels.size(0)
    correct+=(predicted==batch_labels).sum().item()

  print(f"Accuracy on test set: {100*correct/total}%")

Accuracy on test set: 87.8345498783455%


In [26]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in trainloader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(100*correct/total)

88.72668288726683
